In [14]:
from pydantic import BaseModel
from typing import Optional

from rdflib import URIRef, BNode, Literal, Namespace, Graph

In [5]:
class BfElement(BaseModel):
    value: str
    label: str

In [6]:
class BfWork(BaseModel):
    resourceType: list[BfElement]

In [7]:
req = {
    "resourceType": [
        {
            "value": "Work",
            "label": "Obra"
        },
        {
            "value": "Text",
            "label": "Texto"
        }
    ]
}

In [8]:
request = BfWork(**req)
request

BfWork(resourceType=[BfElement(value='Work', label='Obra'), BfElement(value='Text', label='Texto')])

In [86]:
for i in request.resourceType:
    print(i.value)

Work
Text


In [48]:
from rdflib import Graph, ConjunctiveGraph, URIRef, Literal, BNode
from rdflib.namespace import RDF, RDFS

BF = Namespace("http://id.loc.gov/ontologies/bibframe/")
g = Graph()
g.bind('bf', BF)

uri = 'https://bibliokeia/works/10'
work = URIRef(uri)

g.add((work, RDF.type, BF.Work))
g.add((work, RDF.type, BF.Text))






@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .

<https://bibliokeia/works/10> a bf:Text,
        bf:Work .




In [21]:
g.serialize(destination='test_work.ttl')

<Graph identifier=N62cd2d1b89ec4c27974603c64c56fd87 (<class 'rdflib.graph.Graph'>)>

In [23]:
s = g.serialize(format='ttl')

In [42]:
from pyfuseki import FusekiUpdate

fuseki = FusekiUpdate("http://localhost:3030" , 'bk') 

In [59]:
print(g.serialize(format='ttl'))

@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .

<https://bibliokeia/works/10> a bf:Text,
        bf:Work .




In [60]:
x = g.serialize(format='ttl')
x

'@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .\n\n<https://bibliokeia/works/10> a bf:Text,\n        bf:Work .\n\n'

In [66]:
s = x.split("\n")

In [80]:
graph = "\n ".join(s[2:])
print(graph)

<https://bibliokeia/works/10> a bf:Text,
         bf:Work .
 
 


In [81]:
graph = f"""
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>  

    INSERT DATA {{
        GRAPH <https://bibliokeia/works/10> 
        {{
            {graph} }} }} """

print(graph)


    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>  

    INSERT DATA {
        GRAPH <https://bibliokeia/works/10> 
        {
            <https://bibliokeia/works/10> a bf:Text,
         bf:Work .
 
  } } 


In [83]:
response = fuseki.run_sparql(graph) 
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [31]:
print(sparql)

INSERT DATA {
        GRAPH https://bibliokeia/works/10
        { @prefix bf: <http://id.loc.gov/ontologies/bibframe/> .

<https://bibliokeia/works/10> a bf:Text,
        bf:Work .

 } }
